# Webscraping Notebook

## Installs

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
from bs4 import BeautifulSoup

## Scrape list of books and urls 

In [7]:
page="1"
url="https://www.goodreads.com/shelf/show/fiction?page=" + page
url

'https://www.goodreads.com/shelf/show/fiction?page=1'

In [46]:
response=requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [50]:
book_titles = [book.text.strip() for book in soup.find_all('a', class_='bookTitle')]

In [51]:
len(book_titles)

50

In [52]:
book_titles[1]

'To Kill a Mockingbird (Paperback)'

In [58]:
tkamb_title = book_titles[1]

In [53]:
book_urls = ['https://www.goodreads.com' + book['href'] for book in soup.find_all('a', class_='bookTitle')]

In [54]:
len(books)

50

In [55]:
books[1]

'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird'

## Get top reviews for each book

In [23]:
response=requests.get(books[1])
soup = BeautifulSoup(response.content, "html.parser")

In [24]:
type(soup)

bs4.BeautifulSoup

In [25]:
len(soup.find_all('section', 'ReviewText'))

30

In [29]:
len(soup.find_all('article', 'ReviewCard'))

30

In [30]:
review_containers = soup.find_all('article', 'ReviewCard')
reviews = []
for container in review_containers:
    review_text = container.find('section', 'ReviewText').text.strip()
    reviews.append(review_text)

In [31]:
len(reviews)

30

In [32]:
type(reviews[1])

str

In [34]:
reviews[0]

'/// gentle reminder that this is not the time to read this book ///This is my first re-read of 2017, and I don\'t regret it one bit. When I first read this book three years ago, I really liked it. Sadly, I didn\'t write my thoughts down in an elaborate way back in the day, but I know for sure, that I didn\'t read critically then. Upon my re-read of this book, I honestly don\'t have good things to say. I am aware that some of my criticism is not a critique of the book itself, but about its perception, and how it is, up to this day, held up as the one true book about race relations in the United States of America.And that really infiruates me. This book was written by a white woman, from a white perspective, about white characters, for a white audience. This book is a pat on the back for the white middle class. This book gives comfort to the white middle class. Comfort that they, especially back in the 1960s, didn\'t need, and allow me to be so bold, didn\'t deserve. Harper Lee\'s focus

In [44]:
tkamb_reviews = reviews.copy()

## Get the average rating

In [36]:
rating = soup.find('div', 'RatingStatistics__column')["aria-label"]

In [37]:
rating

'Average rating of 4.27 stars.'

In [40]:
float(rating[18:22])

4.27

In [43]:
tkamb_rating = float(rating[18:22])

## Saving title, rating, and 30 reviews together in a class

In [57]:
class BookInfo:
    def __init__(self, rating, reviews, title):
        self.title = title
        self.rating = rating
        self.reviews = reviews
    
    def get_title(self):
        return self.title
    def get_rating(self):
        return self.rating
    def get_reviews(self):
        return self.reviews

In [59]:
book1 = BookInfo(rating=tkamb_rating, reviews=tkamb_reviews, title=tkamb_title)

In [61]:
book1.get_rating()

4.27